# Add Attendance Points Script
**Ben Paulson -- 9/12/2023**<br>

Given a specific attendance-points form (same format), can parse and add the points to each student that filled out the form. Warnings will be provided for students that are not already included in the form or if there was a typo.<br>

A format to follow is the following W2 attendance form from 2023: https://forms.office.com/Pages/DesignPageV2.aspx?prevorigin=Marketing&origin=NeoPortalPage&subpage=design&id=rM5GQNP9yUasgLfEpJurcGAyFplwhXJCtqB2wsxmGVlUNERLT0g1N0IzOU9aVk1INjE5S0w5VjBRQS4u

# Part 1: Import Statements

In [14]:
!pip install openpyxl # Able to parse excel files using pandas

ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
import os

# Part 2: Parsing the Provided CSV Files
Files in `to_parse` folder will be analyzed and points will be added. Once the sheet is analyzed, will be deleted to save AWS storage costs

In [2]:
attendance_df = {} # {file_name: df}

# Read through all files in the `to_parse` folder
directory_name = './to_parse'
for file in os.listdir(directory_name):
    if file.endswith('.xlsx'):
        file_name = directory_name + '/' + file
        attendance_df[file_name] = pd.read_excel(file_name)

In [3]:
website_data = pd.read_csv('../../data/User_Data.csv')

In [4]:
pd.notna(website_data.loc[100, 'Awards'])

False

In [11]:
# Create the member_list.csv
# df = website_data
# df['First Name'] = df['User'].str.split(' ').str[0]
# df['Last Name'] = df['User'].str.split(' ').str[1]
# df = df[['First Name', 'Last Name']]

# # Create a new column called 'email'
# # Email is last_name + first letter of first name + @msoe.edu (all lowercase)
# df['email'] = df['Last Name'] + df['First Name'].str[0] + '@msoe.edu'
# df['email'] = df['email'].str.lower()

# df.to_csv('member_list.csv')

C:\Users\paulsonb\AppData\Local\Temp\ipykernel_8848\2155176163.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email'] = df['Last Name'] + df['First Name'].str[0] + '@msoe.edu'
C:\Users\paulsonb\AppData\Local\Temp\ipykernel_8848\2155176163.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['email'] = df['email'].str.lower()


In [7]:
for file_name,df in zip(attendance_df.keys(), attendance_df.values()):
    attendance_names = df['First and Last Name'].tolist()
    
    # Print a warning for each of the attendance_names which are not in the 'User' column of the website_data df
    ADD_MISSING_USERS = True
    ADD_BADGE = None
    for name in attendance_names:
        name = name.strip()
        name = name.title()

        # Create the list website_data['User'].tolist() and apply .strip() and .title() to each element
        name_list = website_data['User'].apply(lambda x: x.strip().title()).tolist()
        if name not in name_list:
            print(f'WARNING: {name} is not in the website_data df')
            
            if ADD_MISSING_USERS:
                website_data = pd.concat([website_data, pd.DataFrame({'User': [name], 'Description': ['test description'], 'Awards':[''], 'All-Time Points': [0], 'Current Points': [0]})], ignore_index=True)

        # If name in list and ADD_BADGE isn't None, add the badge to their 'Awards' column
        if ADD_BADGE:
            index = name_list.index(name)
            # If they already have a badge, add the new badge w/ '|' at beginning
            if pd.notna(website_data.loc[index, 'Awards']):
                website_data.loc[index, 'Awards'] += f'|{ADD_BADGE}'
            # If they don't have a badge, add the new badge w/o '|' at beginning
            else:
                website_data.loc[index, 'Awards'] = ADD_BADGE
            

    # Filter out all records in the website_data df where the 'User' column isn't in the attendance_names list
    filtered_website_data = website_data[website_data['User'].isin(attendance_names)]

    # For all the records that are in filtred_website_data, add a 1 to the 'All-Time Points' and 'Current Points' columns in the website_data df
    for index, row in filtered_website_data.iterrows():
        website_data.loc[index, 'All-Time Points'] += 1
        website_data.loc[index, 'Current Points'] += 1
    
    # Save the updated website_data df to a csv file
    website_data.to_csv('../../data/User_Data.csv', index=False)

    # Delete the file once finished parsing
    os.remove(file_name)

In [9]:
# If anyone in website_data has 0 All-Time Points and 0 Current Points (at the same time), make sure they are at least given 1 point
website_data.loc[(website_data['All-Time Points'] == 0) & (website_data['Current Points'] == 0), 'All-Time Points'] = 1
website_data.to_csv('../../data/User_Data.csv', index=False)